In [1]:
!pwd

/content


In [2]:
cd /content/drive/MyDrive/Mesh_tooth_3d/MeshSegNet

/content/drive/MyDrive/Mesh_tooth_3d/MeshSegNet


In [3]:
!pip install vedo

     |████████████████████████████████| 18.9 MB 4.2 MB/s 
     |████████████████████████████████| 59.5 MB 1.3 MB/s 
     |████████████████████████████████| 3.1 MB 54.9 MB/s 
     |████████████████████████████████| 365 kB 81.1 MB/s 
     |████████████████████████████████| 3.6 MB 57.9 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
     |████████████████████████████████| 251 kB 71.8 MB/s 
     |████████████████████████████████| 1.1 MB 63.1 MB/s 
     |████████████████████████████████| 271 kB 79.3 MB/s 
     |████████████████████████████████| 144 kB 77.9 MB/s 
     |████████████████████████████████| 160 kB 79.3 MB/s 
  Created wheel for vedo: filename=vedo-2022.0.1-py3-none-any.whl size=18966813 sha256=eaad66dec53b06dd8d1566741704efcf32632c40219c2a36ba9d2646817779fc
  Stored in directory: /root/.cache/pip/wheels/4d/78/7b/96e04d8011790796bbc8182aac2d4fcfee49654140dee53c72
  Created wheel for autobahn: filename=autobahn-21.11.1-cp37-cp37m-linux_x86_64.whl size=537824 sha256=b7

In [4]:
!pip install open3d

     |████████████████████████████████| 395.5 MB 32 kB/s 
     |████████████████████████████████| 596 kB 62.0 MB/s 
     |████████████████████████████████| 4.3 MB 59.6 MB/s 
     |████████████████████████████████| 1.1 MB 69.8 MB/s 
     |████████████████████████████████| 8.5 MB 61.6 MB/s 
     |████████████████████████████████| 428 kB 79.8 MB/s 
     |████████████████████████████████| 395 kB 81.0 MB/s 
     |████████████████████████████████| 61 kB 11.1 MB/s 
     |████████████████████████████████| 130 kB 82.1 MB/s 
     |████████████████████████████████| 53 kB 2.9 MB/s 
     |████████████████████████████████| 79 kB 11.2 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
  Attempting uninstall: jupyter-client
    Found e

In [5]:
!pip install pygco

  Created wheel for pygco: filename=pygco-0.0.16-cp37-cp37m-linux_x86_64.whl size=291349 sha256=b3598fc6bb9d07fe00eb3513379e34ccbca8a5958cb26b4f27936d0a3f163ebc
  Stored in directory: /root/.cache/pip/wheels/70/ad/c2/200c1bc7ad09996b949956673354313a7e4f52d1b845f29256
Successfully built pygco


In [6]:
!pip install ipyvtklink

     |████████████████████████████████| 253 kB 6.6 MB/s 
     |████████████████████████████████| 130 kB 76.3 MB/s 


In [7]:
import os
import numpy as np
import torch
import torch.nn as nn
from meshsegnet import *
import vedo
import pandas as pd
from losses_and_metrics_for_mesh import *
from scipy.spatial import distance_matrix
import scipy.io as sio
import shutil
import time
from sklearn.neighbors import KNeighborsClassifier
from pygco import cut_from_graph
import open3d as o3d

In [8]:
 #gpu_id = utils.get_avail_gpu()
gpu_id = 0
torch.cuda.set_device(gpu_id) # assign which gpu will be used (only linux works)

upsampling_method = 'KNN'

model_path = './models'
model_name = 'MeshSegNet_Max_15_classes_72samples_lr1e-2_best.zip'

In [9]:
mesh_path = '/content/drive/MyDrive/Mesh_tooth_3d/test_predict_stl_folder'  # need to define
#mesh_path = '/content/drive/MyDrive/Mesh_tooth_3d/sample_vtp'  # need to define
#sample_filenames = ['76543LowerJaw.stl'] # need to define
sample_filenames = ['confirm2.stl'] # need to define
output_path = './outputs'
if not os.path.exists(output_path):
  os.mkdir(output_path)

num_classes = 15
num_channels = 15

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MeshSegNet(num_classes=num_classes, num_channels=num_channels).to(device, dtype=torch.float)

In [11]:
print(model)

MeshSegNet(
  (mlp1_conv1): Conv1d(15, 64, kernel_size=(1,), stride=(1,))
  (mlp1_conv2): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
  (mlp1_bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (mlp1_bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fstn): STNkd(
    (conv1): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (conv3): Conv1d(128, 512, kernel_size=(1,), stride=(1,))
    (fc1): Linear(in_features=512, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=128, bias=True)
    (fc3): Linear(in_features=128, out_features=4096, bias=True)
    (relu): ReLU()
    (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [12]:
 # load trained model
checkpoint = torch.load(os.path.join(model_path, model_name), map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
del checkpoint
model = model.to(device, dtype=torch.float)

#cudnn
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

In [13]:
model.eval()
with torch.no_grad():
  for i_sample in sample_filenames:

    start_time = time.time()
    tmp_path = './.tmp/'
    if not os.path.exists(tmp_path):
      os.makedirs(tmp_path)
    
    
    print('Predicting Sample filename: {}'.format(i_sample))
    mesh = vedo.load(os.path.join(mesh_path, i_sample))
  
      # pre-processing: downsampling
    if mesh.NCells() > 10000:
      print('\tDownsampling...')
      target_num = 10000
      ratio = target_num/mesh.NCells() # calculate ratio
      mesh_d = mesh.clone()
      mesh_d.decimate(fraction=ratio)
      predicted_labels_d = np.zeros([mesh_d.NCells(), 1], dtype=np.int32)
    else:
      mesh_d = mesh.clone()
      predicted_labels_d = np.zeros([mesh_d.NCells(), 1], dtype=np.int32)

    # move mesh to origin
    print('\tPredicting...')
    cells = np.zeros([mesh_d.NCells(), 9], dtype='float32')
    for i in range(len(cells)):
      cells[i][0], cells[i][1], cells[i][2] = mesh_d.polydata().GetPoint(mesh_d.polydata().GetCell(i).GetPointId(0)) # don't need to copy
      cells[i][3], cells[i][4], cells[i][5] = mesh_d.polydata().GetPoint(mesh_d.polydata().GetCell(i).GetPointId(1)) # don't need to copy
      cells[i][6], cells[i][7], cells[i][8] = mesh_d.polydata().GetPoint(mesh_d.polydata().GetCell(i).GetPointId(2)) # don't need to copy

    original_cells_d = cells.copy()

    mean_cell_centers = mesh_d.centerOfMass()
    cells[:, 0:3] -= mean_cell_centers[0:3]
    cells[:, 3:6] -= mean_cell_centers[0:3]
    cells[:, 6:9] -= mean_cell_centers[0:3]

     # customized normal calculation; the vtk/vedo build-in function will change number of points
    v1 = np.zeros([mesh_d.NCells(), 3], dtype='float32')
    v2 = np.zeros([mesh_d.NCells(), 3], dtype='float32')
    v1[:, 0] = cells[:, 0] - cells[:, 3]
    v1[:, 1] = cells[:, 1] - cells[:, 4]
    v1[:, 2] = cells[:, 2] - cells[:, 5]
    v2[:, 0] = cells[:, 3] - cells[:, 6]
    v2[:, 1] = cells[:, 4] - cells[:, 7]
    v2[:, 2] = cells[:, 5] - cells[:, 8]
    mesh_normals = np.cross(v1, v2)
    mesh_normal_length = np.linalg.norm(mesh_normals, axis=1)
    mesh_normals[:, 0] /= mesh_normal_length[:]
    mesh_normals[:, 1] /= mesh_normal_length[:]
    mesh_normals[:, 2] /= mesh_normal_length[:]
    mesh_d.celldata['Normal'] = mesh_normals
 
    # preprae input
    points = mesh_d.points().copy()
    points[:, 0:3] -= mean_cell_centers[0:3]
    normals = mesh_d.celldata['Normal'] # need to copy, they use the same memory address
    barycenters = mesh_d.cellCenters() # don't need to copy
    barycenters -= mean_cell_centers[0:3]

    #normalized data
    maxs = points.max(axis=0)
    mins = points.min(axis=0)
    means = points.mean(axis=0)
    stds = points.std(axis=0)
    nmeans = normals.mean(axis=0)
    nstds = normals.std(axis=0)

    for i in range(3):
      cells[:, i] = (cells[:, i] - means[i]) / stds[i] #point 1
      cells[:, i+3] = (cells[:, i+3] - means[i]) / stds[i] #point 2
      cells[:, i+6] = (cells[:, i+6] - means[i]) / stds[i] #point 3
      barycenters[:,i] = (barycenters[:,i] - mins[i]) / (maxs[i]-mins[i])
      normals[:,i] = (normals[:,i] - nmeans[i]) / nstds[i]

    X = np.column_stack((cells, barycenters, normals))
    #X = (X-np.ones((X.shape[0], 1))*np.mean(X, axis=0)) / (np.ones((X.shape[0], 1))*np.std(X, axis=0))

    # computing A_S and A_L
    A_S = np.zeros([X.shape[0], X.shape[0]], dtype='float32')
    A_L = np.zeros([X.shape[0], X.shape[0]], dtype='float32')
    D = distance_matrix(X[:, 9:12], X[:, 9:12])
    A_S[D<0.1] = 1.0
    A_S = A_S / np.dot(np.sum(A_S, axis=1, keepdims=True), np.ones((1, X.shape[0])))

    A_L[D<0.2] = 1.0
    A_L = A_L / np.dot(np.sum(A_L, axis=1, keepdims=True), np.ones((1, X.shape[0])))

    # numpy -> torch.tensor
    X = X.transpose(1, 0)
    X = X.reshape([1, X.shape[0], X.shape[1]])
    X = torch.from_numpy(X).to(device, dtype=torch.float)
    print("@@@@@@ X.shape")
    print(X.shape)
    A_S = A_S.reshape([1, A_S.shape[0], A_S.shape[1]])
    A_L = A_L.reshape([1, A_L.shape[0], A_L.shape[1]])
    A_S = torch.from_numpy(A_S).to(device, dtype=torch.float)
    A_L = torch.from_numpy(A_L).to(device, dtype=torch.float)

    tensor_prob_output = model(X, A_S, A_L).to(device, dtype=torch.float)
    print(tensor_prob_output.shape)
    patch_prob_output = tensor_prob_output.cpu().numpy()

    for i_label in range(num_classes):
      #print(type(patch_prob_output))
      predicted_labels_d[np.argmax(patch_prob_output[0, :], axis=-1)==i_label] = i_label
      # output downsampled predicted labels
    mesh2 = mesh_d.clone()
    print(type(predicted_labels_d))
    print(predicted_labels_d.shape)
    print(predicted_labels_d)
    mesh2.celldata['Label'] = predicted_labels_d
    vedo.write(mesh2, os.path.join(output_path, '{}_d_predicted.vtp'.format(i_sample[:-4])))

    print('\tRefining by pygco...')
    round_factor = 100
    patch_prob_output[patch_prob_output<1.0e-6] = 1.0e-6

    # unaries
    unaries = -round_factor * np.log10(patch_prob_output)
    unaries = unaries.astype(np.int32)
    unaries = unaries.reshape(-1, num_classes)

    # parawise
    pairwise = (1 - np.eye(num_classes, dtype=np.int32))

    #edges
    normals = mesh_d.getCellArray('Normal').copy() # need to copy, they use the same memory address
    cells = original_cells_d.copy()
    barycenters = mesh_d.cellCenters() # don't need to copy
    cell_ids = np.asarray(mesh_d.faces())

    lambda_c = 30
    edges = np.empty([1, 3], order='C')
    for i_node in range(cells.shape[0]):
    # Find neighbors
      nei = np.sum(np.isin(cell_ids, cell_ids[i_node, :]), axis=1)
      nei_id = np.where(nei==2)
      for i_nei in nei_id[0][:]:
        if i_node < i_nei:
          cos_theta = np.dot(normals[i_node, 0:3], normals[i_nei, 0:3])/np.linalg.norm(normals[i_node, 0:3])/np.linalg.norm(normals[i_nei, 0:3])
          if cos_theta >= 1.0:
            cos_theta = 0.9999
          theta = np.arccos(cos_theta)
          phi = np.linalg.norm(barycenters[i_node, :] - barycenters[i_nei, :])
          if theta > np.pi/2.0:
            edges = np.concatenate((edges, np.array([i_node, i_nei, -np.log10(theta/np.pi)*phi]).reshape(1, 3)), axis=0)
          else:
            beta = 1 + np.linalg.norm(np.dot(normals[i_node, 0:3], normals[i_nei, 0:3]))
            edges = np.concatenate((edges, np.array([i_node, i_nei, -beta*np.log10(theta/np.pi)*phi]).reshape(1, 3)), axis=0)
    edges = np.delete(edges, 0, 0)
    edges[:, 2] *= lambda_c*round_factor
    edges = edges.astype(np.int32)

    refine_labels = cut_from_graph(edges, unaries, pairwise)
    refine_labels = refine_labels.reshape([-1, 1])

    # output refined result
    mesh3 = mesh_d.clone()
    mesh3.addCellArray(refine_labels, 'Label')
    vedo.write(mesh3, os.path.join(output_path, '{}_d_predicted_refined.vtp'.format(i_sample[:-4])))

    # upsampling
    print('\tUpsampling...')
    if mesh.NCells() > 100000:
      target_num = 100000 # set max number of cells
      ratio = target_num/mesh.NCells() # calculate ratio
      mesh.decimate(fraction=ratio)
      print('Original contains too many cells, simpify to {} cells'.format(mesh.NCells()))

    # get fine_cells
    cells = np.zeros([mesh.NCells(), 9], dtype='float32')
    for i in range(len(cells)):
      cells[i][0], cells[i][1], cells[i][2] = mesh.polydata().GetPoint(mesh.polydata().GetCell(i).GetPointId(0)) # don't need to copy
      cells[i][3], cells[i][4], cells[i][5] = mesh.polydata().GetPoint(mesh.polydata().GetCell(i).GetPointId(1)) # don't need to copy
      cells[i][6], cells[i][7], cells[i][8] = mesh.polydata().GetPoint(mesh.polydata().GetCell(i).GetPointId(2)) # don't need to copy

    fine_cells = cells

    barycenters = mesh3.cellCenters() # don't need to copy
    fine_barycenters = mesh.cellCenters() # don't need to copy

    if upsampling_method == 'SVM':
      #clf = SVC(kernel='rbf', gamma='auto', probability=True, gpu_id=gpu_id)
      clf = SVC(kernel='rbf', gamma='auto', gpu_id=gpu_id)
      # train SVM
      #clf.fit(mesh2.cells, np.ravel(refine_labels))
      #fine_labels = clf.predict(fine_cells)

      clf.fit(barycenters, np.ravel(refine_labels))
      fine_labels = clf.predict(fine_barycenters)
      fine_labels = fine_labels.reshape(-1, 1)
    elif upsampling_method == 'KNN':
      neigh = KNeighborsClassifier(n_neighbors=3)
      # train KNN
      #neigh.fit(mesh2.cells, np.ravel(refine_labels))
                #fine_labels = neigh.predict(fine_cells)

      neigh.fit(barycenters, np.ravel(refine_labels))
      fine_labels = neigh.predict(fine_barycenters)
      fine_labels = fine_labels.reshape(-1, 1)

    mesh.addCellArray(fine_labels, 'Label')
    vedo.write(mesh, os.path.join(output_path, '{}_predicted_refined.vtp'.format(i_sample[:-4])))

    #remove tmp folder
    shutil.rmtree(tmp_path)

    end_time = time.time()
    print('Sample filename: {} completed'.format(i_sample))
    print('\tcomputing time: {0:.2f} sec'.format(end_time-start_time))    


    

Predicting Sample filename: confirm2.stl
	Predicting...
@@@@@@ X.shape
torch.Size([1, 15, 9899])
torch.Size([1, 9899, 15])
<class 'numpy.ndarray'>
(9899, 1)
[[2]
 [2]
 [2]
 ...
 [0]
 [0]
 [0]]
	Refining by pygco...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:131: DeprecationWarning: Call to deprecated method getCellArray. (Please use myobj.celldata[name] instead.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:163: DeprecationWarning: Call to deprecated method addCellArray. (Please use myobj.celldata[name] = myarr instead.)


	Upsampling...
Sample filename: confirm2.stl completed
	computing time: 15.07 sec


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:206: DeprecationWarning: Call to deprecated method addCellArray. (Please use myobj.celldata[name] = myarr instead.)


In [14]:
#후처리 전 
confirm = mesh2.getCellArray()
  
data_df3 = pd.DataFrame(mesh2.celldata['Label'])
data_df3.value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated method getCellArray. (Please use myobj.celldata[name] instead.)
  


0     4221
13     705
2      579
3      477
12     464
1      417
14     387
7      362
11     348
8      344
10     343
6      342
5      328
4      328
9      254
dtype: int64

In [15]:
#후처리 후 
new_data_df3 = pd.DataFrame(mesh.celldata['Label'])
new_data_df3.value_counts()

0     3880
2      561
13     553
14     508
11     505
1      467
4      445
10     417
8      398
3      397
5      379
7      371
12     360
6      335
9      323
dtype: int64

In [17]:
#new_data_df3.columns = ['Label']
#new_data_df3

In [18]:
#mesh2.celss()
#mesh2.points()
#mesh2.celldata['Label']

In [19]:
# print(type(mesh2.celldata[0][1]))
# mesh2.celldata[0][0:10]

In [20]:
print(type(mesh2.cells()))

<class 'list'>


In [21]:
#현재 까지 파악한 정보로는 그면에 라벨이 들어간것 


In [22]:
mesh2.NPoints()

5067

In [23]:
mesh2.NCells()

9899

In [36]:
# ##이제 이것을 알고리즘 및 함수화 할 필요가 있음 
# ##또한 학습을 시키면 먼가 달라지지 않을까? 
# ##애초에 파이썬에서 시각화를 하는 방법을 찾아도 좋다 

# def get_label_point(value):
#   result = pd.concat([new_data_df3, new_data_df4], axis = 1) 
#   testdf = result[result['Label'] == value]
#   parsing_df = testdf[[0,1,2]]
#   parsing_list = parsing_df.values.tolist()
#   #print(parsing_list)
#   answer = sum(parsing_list, [])
#   #print(set(answer))
#   answer = set(answer)
#   #print(len(answer))
#   result_df = pd.DataFrame()
#   new_df = pd.DataFrame()
#   for point in answer:
#     new_df = data_df6.iloc[np.where(data_df6.index == point)]
#     #print(new_df)
#     result_df = result_df.append(new_df)
#     result_df.to_csv(f'./new_Label{value}_check.txt', sep = ' ',index = False,header=None)
#     result_df


In [ ]:
# for i in range(15):
#   get_label_point(i)

면 추출 시도


In [37]:
new_data_df3

,Label
0,2
1,2
2,2
3,2
4,2
...,...
9894,0
9895,0
9896,0
9897,0


In [38]:
new_data_df4

,0,1,2
0,0,1,2
1,2,1,3
2,1,4,3
3,5,1,0
4,4,1,6
...,...,...,...
9894,5065,864,836
9895,818,814,865
9896,865,814,816
9897,5064,840,839


In [103]:
# import open3d as o3d
# p3d = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(mesh.points()), 
#                                 o3d.utility.Vector3iVector(test_face_df[[0,1,2]].to_numpy()))
# print(p3d)
# #인자로 numpy를 주면된다. 
# o3d.io.write_triangle_mesh("./data.ply",p3d)
# print('!!!!')

TriangleMesh with 5067 points and 3880 triangles.
!!!!


In [24]:
def get_result_ply(value):
  new_data_df3 = pd.DataFrame(mesh.celldata['Label'])
  new_data_df4 = pd.DataFrame(mesh.cells())
  new_data_df3.columns = ['Label']
  face_extract = pd.concat([new_data_df3, new_data_df4], axis = 1) 
  test_face_df = face_extract[face_extract['Label'] == value]
  p3d = o3d.geometry.TriangleMesh(o3d.utility.Vector3dVector(mesh.points()), 
                                o3d.utility.Vector3iVector(test_face_df[[0,1,2]].to_numpy()))
  o3d.io.write_triangle_mesh(f"./predict_{value}_result.ply",p3d)



In [25]:
for i in range(15):
  get_result_ply(i)

INFO - 2022-01-21 13:40:52,564 - utils - NumExpr defaulting to 2 threads.
